<a href="https://colab.research.google.com/github/L-ENA/EvaluationSynthesisMethods/blob/main/Paper_UNICEF_Lena_multi_class_label_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install Keras-Preprocessing

import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score, recall_score,precision_score
import pickle

from google.colab import drive
from transformers import BertTokenizer,XLNetTokenizer,RobertaTokenizer,AutoTokenizer
from transformers import BertForSequenceClassification, XLNetForSequenceClassification, RobertaForSequenceClassification, AutoModelForSequenceClassification
from transformers import AdamW
import datetime
import os
from tqdm.auto import tqdm
from tqdm import trange
from ast import literal_eval

# The best trained model is available for download here:

# https://drive.google.com/drive/folders/1-0VXJcY_GKBNq6-5GprPvwdnTc4Raud3?usp=sharing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 8.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.9 MB/s eta 0:00:00


In [ ]:
######COLAB STUFF
drive.mount('/content/drive')

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found GPU at: /device:GPU:0


'Tesla T4'

In [ ]:
#############<<<<<<<<<<<<<<<<<<<<<<<<<parameters/args
#################################################Numbers, seeds, etc
max_length = 100#tokens in tokenizer
batch_size = 8
max_samples_eval=800000
max_samples_train=800000
answer_to_everything=42#ie the random seed
threshold = 0.2
train_splits=0.4

epochs = 3#epochs
stratify_one_freq=True#to split inputs with one label only and to exclusively add them to training data (so that multilabel data are distributed fairly).
dev_fraction=0.00#how may data to keep for dev
divider=200#number of steps after which to do dev eval during training
##########################################################################paths

#model_name='bert-base-uncased'
#model_name='scibert'
model_name='specter'
#model_name='BERT tiny'
#model_name="xlnet"

dataset='unicef'



if dataset=="unicef":#read the spreadsheet created from the method in the other colab notebook and split the data according to split parameter above
  base_path="/content/drive/MyDrive/Colab Notebooks/unicef_data"

  #df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/unicef_data/train_UNICEF_initial_reduced.csv').fillna("")
  df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/unicef_data/train_UNICEF_new.csv').fillna("")

  train_path=os.path.join(base_path,'train_unicef.csv')

  test_path=os.path.join(base_path,'test_unicef.csv')

  train=df.sample(frac=train_splits,random_state=answer_to_everything) #random state is a seed value
  test=df.drop(train.index)
  train.to_csv(train_path,index=False)
  test.to_csv(test_path,index=False)



output_path="/content/drive/MyDrive/Colab Notebooks/unicef_results"

log_path=os.path.join(output_path, "{}_{}_log.txt".format(dataset,model_name))
train_data_loader=os.path.join(output_path,'{}_{}_train_data_loader'.format(dataset,model_name))
validation_data_loader=os.path.join(output_path,'{}_{}_validation_data_loader'.format(dataset,model_name))


model_saved=os.path.join(output_path,'{}_{}_pymodel_tuned_best/'.format(dataset,model_name))
if not os.path.exists(model_saved):
  os.mkdir(model_saved)



#######################################load data frames################################################################################################

try:
  df = pd.read_csv(train_path).sample(max_samples_train, random_state=answer_to_everything)
except:
  df = pd.read_csv(train_path)

df = df.sample(frac=1, random_state=answer_to_everything).reset_index(drop=True) #shuffle rows
cols = df.columns
label_cols = list(cols[2:])#the number of columns that actually include labels
num_labels = len(label_cols)


test_df = pd.read_csv(test_path)


test_label_cols = list(test_df.columns[2:])
try:
  test_df = test_df[~test_df[test_label_cols].eq(-1).any(axis=1)].sample(max_samples_eval,random_state=answer_to_everything) #remove irrelevant rows/comments with -1 values

except:
  test_df = test_df[~test_df[test_label_cols].eq(-1).any(axis=1)]

log=[]
log.append(datetime.datetime.now())
log.append("Model {} on dataset {} with batch size {}.\nNum train: {}; num eval: {}.\nSeed: {}; Length: {}; Threshold: {}".format(model_name, dataset,batch_size,max_samples_train,max_samples_eval,answer_to_everything,max_length, threshold))

print(test_label_cols)

['enablers', 'barriers', 'lessons', 'recommendation', 'background']


In [ ]:
################################MODELS##############################################
#
#!!!!!!!!!!!!MAKE SURE THEY EXIST IN HUGGINGFACE AND THAT THEY MATCH
##################################

if model_name=='bert-base-uncased':
  tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True) # tokenizer
  model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

elif model_name=='BERT tiny':

  tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-mini', do_lower_case=True) # tokenizer
  model = BertForSequenceClassification.from_pretrained('prajjwal1/bert-mini', num_labels=num_labels)


elif model_name=='scibert':
  tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
  model = AutoModelForSequenceClassification.from_pretrained('allenai/scibert_scivocab_uncased', num_labels=num_labels)


elif model_name=='xlnet':
  tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
  model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels=num_labels)

elif model_name=='specter':

  tokenizer = AutoTokenizer.from_pretrained("allenai/specter")

  #model = AutoModel.from_pretrained("allenai/specter")
  model = AutoModelForSequenceClassification.from_pretrained("allenai/specter", num_labels=num_labels)


model.cuda()

#XLNetForSequenceClassification
########################MODEL SETUP
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

#see comments for previous parameters. Adjust parameters for different models. Note, in the code below we are also simply loading a saved version of the best model for our dataset so make sure code is adjusted for retraining throughout
optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5,correct_bias=True)#optimizer = AdamW(optimizer_grouped_parameters,lr=2e-6,correct_bias=True)# optimizer = AdamW(model.parameters(),lr=2e-5)  # Default optimization
# setting custom optimization parameters. You may implement a scheduler here as well.

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.weight', 'logits_proj.bias', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Setting custom optimization parameters for the AdamW optimizer https://huggingface.co/transformers/main_classes/optimizer_schedules.html

## Load and Preprocess Training Data

Dataset statistics are shown and it will be tokenized and so on

In [ ]:
##READ AND STATS


log.append("<<<<<<<<<<<<<<<<<<<<TRAIN DATA STATS>>>>>>>>>>>>>>>>>>>")
log.append(df.shape)
print(df.shape)
log.append('Unique comments: {}'.format(df.comment_text.nunique() == df.shape[0]))
log.append('Null values: {}'.format(df.isnull().values.any()))
# df[df.isna().any(axis=1)]
log.append('average sentence length: {}'.format(df.comment_text.str.split().str.len().mean()))
log.append('stdev sentence length: {}'.format(df.comment_text.str.split().str.len().std()))

###Distribution of data

log.append('Label columns: {}'.format(label_cols))
log.append('Count of 1 (pos. class) per label: \n{}\n'.format(df[label_cols].sum())) # Label counts, may need to downsample or upsample
log.append('Count of 0 (neg. class) per label: \n{}\n'.format(df[label_cols].eq(0).sum()))

log.append("<<<<<<<<<<<<<<<<<<<<<<<TEST DATA STATS>>>>>>>>>>>>>>>>>>>")


print(test_df.shape)

log.append('Unique comments: {}'.format(test_df.comment_text.nunique() == test_df.shape[0]))
log.append('Null values: {}'.format(test_df.isnull().values.any())) #should not be any null sentences or labels
log.append('Same columns between train and test: {}'.format(label_cols == test_label_cols)) #columns should be the same
test_df.head()

log.append('average sentence length: {}'.format(test_df.comment_text.str.split().str.len().mean()))
log.append('stdev sentence length: {}'.format(test_df.comment_text.str.split().str.len().std()))

###Distribution of data

num_labels = len(test_label_cols )
log.append('Label columns: {}'.format(test_label_cols ))
log.append('Count of 1 (pos. class) per label: \n{}\n'.format(test_df[test_label_cols ].sum())) # Label counts, may need to downsample or upsample
log.append('Count of 0 (neg. class) per label: \n{}\n'.format(test_df[test_label_cols ].eq(0).sum()))



test_df['one_hot_labels'] = list(test_df[test_label_cols].values)
test_df.head()
log.append(test_df.shape[0])




(346, 7)
(519, 7)


In [ ]:
log

[datetime.datetime(2023, 3, 22, 14, 3, 8, 971729),
 'Model xlnet on dataset unicef with batch size 8.\nNum train: 800000; num eval: 800000.\nSeed: 42; Length: 100; Threshold: 0.2',
 '<<<<<<<<<<<<<<<<<<<<TRAIN DATA STATS>>>>>>>>>>>>>>>>>>>',
 (346, 7),
 'Unique comments: False',
 'Null values: True',
 'average sentence length: 31.572254335260116',
 'stdev sentence length: 33.94414395895322',
 "Label columns: ['enablers', 'barriers', 'lessons', 'recommendation', 'background']",
 'Count of 1 (pos. class) per label: \nenablers           19.0\nbarriers           21.0\nlessons            68.0\nrecommendation     78.0\nbackground        157.0\ndtype: float64\n',
 'Count of 0 (neg. class) per label: \nenablers          327\nbarriers          325\nlessons           277\nrecommendation    268\nbackground        189\ndtype: int64\n',
 '<<<<<<<<<<<<<<<<<<<<<<<TEST DATA STATS>>>>>>>>>>>>>>>>>>>',
 'Unique comments: False',
 'Null values: False',
 'Same columns between train and test: True',
 'avera

In [ ]:
##########PROCESS AND ENCODE TRAIN DATA
df['one_hot_labels'] = list(df[label_cols].values)#one-hot for multiclass stuff
labels = list(df.one_hot_labels.values)
comments = list(df.comment_text.values)
df.head()
encodings = tokenizer.batch_encode_plus(comments,max_length=max_length,pad_to_max_length=True, truncation=True) # tokenizer's encoding method
print('tokenizer outputs: ', encodings.keys())
input_ids = encodings['input_ids'] # tokenized and encoded sentences
token_type_ids = encodings['token_type_ids'] # token type ids
attention_masks = encodings['attention_mask'] # attention masks

if stratify_one_freq and dev_fraction>0.0:
  # Identifying indices of 'one_hot_labels' entries that only occur once - this will allow us to stratify split our training data later
  label_counts = df.one_hot_labels.astype(str).value_counts()
  one_freq = label_counts[label_counts==1].keys()
  one_freq_idxs = sorted(list(df[df.one_hot_labels.astype(str).isin(one_freq)].index), reverse=True)
  print('df label indices with only one instance: ', one_freq_idxs)
  # Gathering single instance inputs to force into the training set after stratified split
  one_freq_input_ids = [input_ids.pop(i) for i in one_freq_idxs]
  one_freq_token_types = [token_type_ids.pop(i) for i in one_freq_idxs]
  one_freq_attention_masks = [attention_masks.pop(i) for i in one_freq_idxs]
  one_freq_labels = [labels.pop(i) for i in one_freq_idxs]
  log.append("Stratification: Number of multilabel entries: {}".format(len(input_ids)))



tokenizer outputs:  dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:


# Use train_test_split to split our data further into dev set, I didn't do it in the unicef project due to extremely low amount of labelled data
if dev_fraction>0.0:
  train_inputs, validation_inputs, train_labels, validation_labels, train_token_types, validation_token_types, train_masks, validation_masks = train_test_split(input_ids, labels, token_type_ids,attention_masks,
                                                            random_state=answer_to_everything, test_size=dev_fraction, stratify = labels)
else:
  train_masks=attention_masks
  train_token_types=token_type_ids
  train_inputs=input_ids
  train_labels=labels

if stratify_one_freq and dev_fraction>0.0:
  # Add one frequency data to train data
  train_inputs.extend(one_freq_input_ids)
  train_labels.extend(one_freq_labels)
  train_masks.extend(one_freq_attention_masks)
  train_token_types.extend(one_freq_token_types)

# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
train_token_types = torch.tensor(train_token_types)

if dev_fraction>0.0:
  validation_inputs = torch.tensor(validation_inputs)
  validation_labels = torch.tensor(validation_labels)
  validation_masks = torch.tensor(validation_masks)
  validation_token_types = torch.tensor(validation_token_types)


In [ ]:


num_batches=int(df.shape[0]/batch_size)+1
log.append("{} batches available as train+dev".format(num_batches))

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop,
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels, train_token_types)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

if dev_fraction>0.0:
  validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels, validation_token_types)
  validation_sampler = SequentialSampler(validation_data)
  validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)
  torch.save(validation_dataloader,validation_data_loader)

In [ ]:

torch.save(train_dataloader,train_data_loader)

## Train Model

In [ ]:

# Store our loss and accuracy for plotting
train_loss_set = []

dev_recall=[]
dev_f1=[]
dev_precision=[]
dev_flat_accuracy=[]


model = model.to(device)

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  ###################################################################################################INITIAL EVAL
  ##############################################################################initial eval before training

  # Training

  # Set our model to training mode (as opposed to evaluation mode)
  model.train()

  # Tracking variables
  tr_loss = 0 #running loss
  nb_tr_examples, nb_tr_steps = 0, 0

  # Train the data for one epoch
  for step, batch in tqdm(enumerate(train_dataloader), total=num_batches):

    if step%divider==0 and dev_fraction>0.0:
      # Put model in evaluation mode to evaluate loss on the validation set
      model.eval()

      # Variables to gather full output
      logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

      # Predict
      for i, batch in enumerate(validation_dataloader):
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels, b_token_types = batch
        with torch.no_grad():
          # Forward pass
          outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          b_logit_pred = outs[0]
          pred_label = torch.sigmoid(b_logit_pred)

          b_logit_pred = b_logit_pred.detach().cpu().numpy()
          pred_label = pred_label.to('cpu').numpy()
          b_labels = b_labels.to('cpu').numpy()

        tokenized_texts.append(b_input_ids)
        logit_preds.append(b_logit_pred)
        true_labels.append(b_labels)
        pred_labels.append(pred_label)

      # Flatten outputs
      pred_labels = [item for sublist in pred_labels for item in sublist]
      true_labels = [item for sublist in true_labels for item in sublist]

      # Calculate Accuracy

      pred_bools = [pl>threshold for pl in pred_labels]
      true_bools = [tl==1 for tl in true_labels]

      dev_f1.append(f1_score(true_bools,pred_bools,average='weighted')*100)
      dev_recall.append(recall_score(true_bools,pred_bools,average='weighted')*100)
      dev_precision.append(precision_score(true_bools,pred_bools,average='weighted')*100)
      dev_flat_accuracy.append(accuracy_score(true_bools, pred_bools)*100)
      # Set our model to training mode (as opposed to evaluation mode)
      model.train()#back to train mode
    ###########################################################################################################



    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()

    # # Forward pass for multiclass classification
    # outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    # loss = outputs[0]
    # logits = outputs[1]

    # Forward pass for multilabel classification
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]
    loss_func = BCEWithLogitsLoss()
    loss = loss_func(logits.view(-1,num_labels),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    # loss_func = BCELoss()
    # loss = loss_func(torch.sigmoid(logits.view(-1,num_labels)),b_labels.type_as(logits).view(-1,num_labels)) #convert labels to float for calculation
    train_loss_set.append(loss.item())

    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # scheduler.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  log.append("Epoch train loss: {}".format(tr_loss/nb_tr_steps))

  model.save_pretrained(model_saved)
  # model = AutoModelForSequenceClassification.from_pretrained(model_saved,num_labels=num_labels)
  # model.cuda()
  #torch.save(model.state_dict(), model_saved)

###############################################################################

# FINAL Validation


# Put model in evaluation mode to evaluate loss on the validation set
if dev_fraction>0.0:
  model.eval()

  # Variables to gather full output
  logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

  # Predict
  for i, batch in enumerate(validation_dataloader):
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    with torch.no_grad():
      # Forward pass
      outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      b_logit_pred = outs[0]
      pred_label = torch.sigmoid(b_logit_pred)

      b_logit_pred = b_logit_pred.detach().cpu().numpy()
      pred_label = pred_label.to('cpu').numpy()
      b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)

  # Flatten outputs
  pred_labels = [item for sublist in pred_labels for item in sublist]
  true_labels = [item for sublist in true_labels for item in sublist]

  # Calculate Accuracy

  pred_bools = [pl>threshold for pl in pred_labels]
  true_bools = [tl==1 for tl in true_labels]

  dev_f1.append(f1_score(true_bools,pred_bools,average='weighted')*100)
  dev_recall.append(recall_score(true_bools,pred_bools,average='weighted')*100)
  dev_precision.append(precision_score(true_bools,pred_bools,average='weighted')*100)
  dev_flat_accuracy.append(accuracy_score(true_bools, pred_bools)*100)

  # print('F1 Validation Accuracy: ', val_f1_accuracy)
  # print('Flat Validation Accuracy: ', val_flat_accuracy)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

Epoch:  33%|███▎      | 1/3 [00:10<00:20, 10.12s/it]

  0%|          | 0/44 [00:00<?, ?it/s]

Epoch:  67%|██████▋   | 2/3 [00:19<00:09,  9.86s/it]

  0%|          | 0/44 [00:00<?, ?it/s]

Epoch: 100%|██████████| 3/3 [00:30<00:00, 10.32s/it]


In [ ]:
# from transformers import TextClassificationPipeline
# pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)
# pipe("I love this movie!")

## Load and Preprocess Test Data

In [ ]:

# Gathering input data
test_labels = list(test_df.one_hot_labels.values)
test_comments = list(test_df.comment_text.values)
# Encoding input data
test_encodings = tokenizer.batch_encode_plus(test_comments,max_length=max_length,pad_to_max_length=True, truncation=True)
test_input_ids = test_encodings['input_ids']
test_token_type_ids = test_encodings['token_type_ids']
test_attention_masks = test_encodings['attention_mask']
# Make tensors out of data
test_inputs = torch.tensor(test_input_ids)
test_labels = torch.tensor(test_labels)
test_masks = torch.tensor(test_attention_masks)
test_token_types = torch.tensor(test_token_type_ids)
# Create test dataloader
test_data = TensorDataset(test_inputs, test_masks, test_labels, test_token_types)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
# Save test dataloader
torch.save(test_dataloader,'test_data_loader')


In [ ]:
print(log)

[datetime.datetime(2023, 3, 22, 14, 3, 8, 971729), 'Model xlnet on dataset unicef with batch size 8.\nNum train: 800000; num eval: 800000.\nSeed: 42; Length: 100; Threshold: 0.2', '<<<<<<<<<<<<<<<<<<<<TRAIN DATA STATS>>>>>>>>>>>>>>>>>>>', (346, 7), 'Unique comments: False', 'Null values: True', 'average sentence length: 31.572254335260116', 'stdev sentence length: 33.94414395895322', "Label columns: ['enablers', 'barriers', 'lessons', 'recommendation', 'background']", 'Count of 1 (pos. class) per label: \nenablers           19.0\nbarriers           21.0\nlessons            68.0\nrecommendation     78.0\nbackground        157.0\ndtype: float64\n', 'Count of 0 (neg. class) per label: \nenablers          327\nbarriers          325\nlessons           277\nrecommendation    268\nbackground        189\ndtype: int64\n', '<<<<<<<<<<<<<<<<<<<<<<<TEST DATA STATS>>>>>>>>>>>>>>>>>>>', 'Unique comments: False', 'Null values: False', 'Same columns between train and test: True', 'average sentence len

## Load model and evaluate it

In [ ]:
# Test

####FYI this is me loading the best final modela pplied in the project.

# The best trained model is available for download here:

# https://drive.google.com/drive/folders/1-0VXJcY_GKBNq6-5GprPvwdnTc4Raud3?usp=sharing


# tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
# model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/unicef_results/eblrb_unicef_specter_pymodel_tuned_best', num_labels=num_labels)
# model.cuda()
# model.to(device)

model = AutoModelForSequenceClassification.from_pretrained(model_saved,num_labels=num_labels)
model.cuda()
model.to(device)


# Put model in evaluation mode to evaluate loss on the validation set
model.eval()

#track variables
logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]

num_batches=int(max_samples_eval/batch_size)+1

# Predict
for i, batch in tqdm(enumerate(test_dataloader),total=num_batches):
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels, b_token_types = batch
  with torch.no_grad():
    # Forward pass
    outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    b_logit_pred = outs[0]
    pred_label = torch.sigmoid(b_logit_pred)


    b_logit_pred = b_logit_pred.detach().cpu().numpy()
    pred_label = pred_label.to('cpu').numpy()
    b_labels = b_labels.to('cpu').numpy()

  tokenized_texts.append(b_input_ids)
  logit_preds.append(b_logit_pred)
  true_labels.append(b_labels)
  pred_labels.append(pred_label)

# Flatten outputs
tokenized_texts = [item for sublist in tokenized_texts for item in sublist]
pred_labels = [item for sublist in pred_labels for item in sublist]

true_labels = [item for sublist in true_labels for item in sublist]
# Converting flattened binary values to boolean values
true_bools = [tl==1 for tl in true_labels]

  0%|          | 0/100001 [00:00<?, ?it/s]

We need to threshold our sigmoid function outputs which range from [0, 1]. Below I use 0.50 as a threshold.

In [ ]:
pred_bools = [pl>0.2 for pl in pred_labels] #boolean output after thresholding

# Print and save classification report
# print('Test F1 Accuracy: ', f1_score(true_bools, pred_bools,average='micro'))
# print('Test Flat Accuracy: ', accuracy_score(true_bools, pred_bools),'\n')
# log.append("DEV PRECISION:")
# log.append(dev_precision)
# log.append("DEV RECALL:")
# log.append(dev_recall)
# log.append("DEV F1:")
# log.append(dev_f1)

clf_report = classification_report(true_bools,pred_bools,target_names=test_label_cols)

# log.append(clf_report)
# log.append(test_label_cols)

# true_ints=np.array(true_bools)
# pred_ints=np.array(pred_bools)
# print(true_ints)
# conf=multilabel_confusion_matrix(true_ints,pred_ints,labels=test_label_cols)
# log.append(datetime.datetime.now())
# log=[str(l)+"\n" for l in log]

# with open(log_path, 'w') as f:
#   f.writelines(log)
#pickle.dump(clf_report, open('classification_report.txt','wb')) #save report
print(clf_report)

                precision    recall  f1-score   support

      enablers       0.00      0.00      0.00        51
      barriers       0.00      0.00      0.00        45
       lessons       0.22      0.92      0.36       132
recommendation       0.42      0.08      0.14       177
    background       0.47      1.00      0.64       368

     micro avg       0.37      0.65      0.47       773
     macro avg       0.22      0.40      0.23       773
  weighted avg       0.36      0.65      0.40       773
   samples avg       0.44      0.65      0.51       773



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##Predict on all the Evaluation texts

Either run this code locally and point to the correct folder or upload the text data to your google drive


In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install Keras-Preprocessing



import pandas as pd
import nltk
import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score, recall_score,precision_score
import pickle

from google.colab import drive
from transformers import BertTokenizer,XLNetTokenizer,RobertaTokenizer,AutoTokenizer
from transformers import BertForSequenceClassification, XLNetForSequenceClassification, RobertaForSequenceClassification, AutoModelForSequenceClassification
from transformers import AdamW
import datetime
import os
from tqdm.auto import tqdm
from tqdm import trange
from ast import literal_eval
nltk.download('punkt')
from google.colab import drive
drive.mount('/content/drive')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 66.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.0 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive


In [ ]:

def remove_errors(l):
    errs = ['\uf0b7', '\ufeff']
    for e in errs:
        l=l.replace(e,"")
    l=l.replace('%PAGEBREAK%', '\n')
    return l

def make_pred_texts(f, outf):
    ids=[]
    texts=[]

    name=f.split('/')[-1].replace('.txt','')
    print(f)
    with open(f,'r', encoding='utf-8', errors='ignore') as ff:
      txt=ff.read()

    # txt=txt.replace('\n\n', '. ')
    # txt=txt.replace('\n', ' ')
    # txt=txt.replace('  ', ' ')

    lines=[remove_errors(l) for l in nltk.tokenize.sent_tokenize(txt)]
    lines=[l for l in lines if len(l) > 20]
    lines=[l for l in lines if '....' not in l]

    refs=['{}_{}'.format(name, i) for i,t in enumerate(lines)]
    ids.extend(refs)
    texts.extend(lines)

    to_predict=pd.DataFrame()
    to_predict['id']=ids
    to_predict['comment_text']=texts
    to_predict['enablers']=0
    to_predict['barriers']=0
    to_predict['lessons']=0
    to_predict['recommendation']=0
    to_predict['background']=0

    to_predict.to_csv(outf, index=False)
    print(len(texts))





def make_batches(test_path, tokenizer):
  max_length = 100#number of tokens in sentence
  batch_size=32



  #test_data_loader='/content/drive/MyDrive/Colab Notebooks/unicef_data/prediction_data_loader'
  test_df = pd.read_csv(test_path).fillna('')
  test_label_cols = list(test_df.columns[2:])

  # print(test_df.shape)

  print('Unique comments: {}'.format(test_df.comment_text.nunique() == test_df.shape[0]))
  print('Null values: {}'.format(test_df.isnull().values.any())) #should not be any null sentences or labels


  print('average sentence length: {}'.format(test_df.comment_text.str.split().str.len().mean()))
  print('stdev sentence length: {}'.format(test_df.comment_text.str.split().str.len().std()))
  num_labels = len(test_label_cols )

  test_df['one_hot_labels'] = list(test_df[test_label_cols].values)#make onehot
  test_labels = list(test_df.one_hot_labels.values)
  test_comments = list(test_df.comment_text.astype(str).values)

  # Encoding input data
  test_encodings = tokenizer.batch_encode_plus(test_comments,max_length=max_length,pad_to_max_length=True,truncation=True)
  test_input_ids = test_encodings['input_ids']


  test_token_type_ids = test_encodings['token_type_ids']
  test_attention_masks = test_encodings['attention_mask']
  # Make tensors out of data
  test_inputs = torch.tensor(test_input_ids)


  test_labels = torch.tensor(test_labels)

  test_masks = torch.tensor(test_attention_masks)
  test_token_types = torch.tensor(test_token_type_ids)
  # print('psst')
  # print(test_labels)
  # Create test dataloader
  test_data = TensorDataset(test_inputs, test_masks, test_labels, test_token_types)

  test_sampler = SequentialSampler(test_data)
  test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
  # Save test dataloader
  #torch.save(test_dataloader,test_data_loader)
  return test_dataloader, num_labels


def predict_sents(test_dataloader, num_labels, model):

  device_name = tf.test.gpu_device_name()
  if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
  print('Found GPU at: {}'.format(device_name))

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  n_gpu = torch.cuda.device_count()
  torch.cuda.get_device_name(0)

  batch_size=32


  #track variables
  logit_preds,true_labels,pred_labels,tokenized_texts = [],[],[],[]



  # Predict
  for i, batch in tqdm(enumerate(test_dataloader)):

    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels, b_token_types = batch
    with torch.no_grad():
      # Forward pass
      outs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      b_logit_pred = outs[0]
      pred_label = torch.sigmoid(b_logit_pred)


      b_logit_pred = b_logit_pred.detach().cpu().numpy()
      pred_label = pred_label.to('cpu').numpy()
      b_labels = b_labels.to('cpu').numpy()

    tokenized_texts.append(b_input_ids)
    logit_preds.append(b_logit_pred)
    true_labels.append(b_labels)
    pred_labels.append(pred_label)


    # Flatten outputs
  tokenized_texts = [item for sublist in tokenized_texts for item in sublist]
  pred_labels = [item for sublist in pred_labels for item in sublist]
    # print('Pred label length:')
    # print(len(pred_labels))
  true_labels = [item for sublist in true_labels for item in sublist]
    # Converting flattened binary values to boolean values
  true_bools = [tl==1 for tl in true_labels]




  return pred_labels

def save_predictions(source_file, pred_labels):
  test_df = pd.read_csv(source_file).fillna('')#semtemces and dummy labels
  test_label_cols = list(test_df.columns[2:])#identify columns where we will add predictions (this list includes their names)
  new_cols=[[] for _ in range(len(test_label_cols))]#store labels first in list of lists because directly updatinf df takes time

  for p in pred_labels:#for each sentence
    for i,proba in enumerate(p):#for each prediction for each class
      new_cols[i].append(proba)#append prediction in correct list (as these lists will become df columns)

  for i, lst in enumerate(new_cols):
    test_df[test_label_cols[i]]=lst#add list in correct place

  outpath=os.path.join('/content/drive/MyDrive/Colab Notebooks/unicef_results/classified_data', source_file.split('/')[-1])
  test_df.to_csv(outpath, index=False)



In [ ]:
##################Get number of labels and all files
num_labels = 5
###############load models

# The best trained model is available for download here, edit path accordingly after saving it:

# https://drive.google.com/drive/folders/1-0VXJcY_GKBNq6-5GprPvwdnTc4Raud3?usp=sharing

tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/unicef_results/eblrb_unicef_specter_pymodel_tuned_best', num_labels=num_labels)
model.cuda()
model.eval()# Put model in evaluation mode to predict


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31116, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
#################################################predict and save predictions for each file
#files=glob.glob('/content/drive/MyDrive/Colab Notebooks/unicef_data/fulltexts/*.txt')#initial batch of files
files=glob.glob('/content/drive/MyDrive/Colab Notebooks/unicef_data/fulltext_additional/*.txt')

problems=[]
for f in tqdm(files):
    outf=f.replace('.txt', '.csv')

    try:
    # if not os.path.exists(os.path.join('/content/drive/MyDrive/Colab Notebooks/unicef_results/classified_data', outf.split('/')[-1])):
      make_pred_texts(f, outf)
      test_dataloader, num_labels= make_batches(outf, tokenizer)
      pred_labels=predict_sents(test_dataloader, num_labels, model)
      save_predictions(outf,pred_labels)
    except:
      problems.append(f)

print('Execution finsished with the following files not being predicted:')
print(problems)
  # else:
  #   print('skip')

# tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
# model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/unicef_results/eblrb_unicef_specter_pymodel_tuned_best', num_labels=num_labels)
# model.cuda()
# model.eval()# Put model out of train mode

In [ ]:
# log.append(clf_report)
# log.append(test_label_cols)
# log.append(datetime.datetime.now())
# log=[str(l)+"\n" for l in log]

# with open(log_path, 'w') as f:
#   f.writelines(log)

##  Filter and create output txtfiles for one class

In [ ]:
import pandas as pd
from glob import glob
from google.colab import drive
drive.mount('/content/drive')
from glob import glob
import pandas  as pd
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

files=glob('/content/drive/MyDrive/Colab Notebooks/unicef_results/classified_data/*')

#############################################for lessons learned and recommendations we are also filtering (ie. removing) and adding top-n sentences matching conditions specified in filter variables below)

new_path='/content/drive/MyDrive/Colab Notebooks/unicef_results/recommendation'
key='recommendation'
filters=['should', 'must', 'recommend']#recomemndations
remove=['lesson learned','lesson learnet','Lesson learnet', 'lessons learned', 'Lesson learned', 'Lessons learned']

# new_path='/content/drive/MyDrive/Colab Notebooks/unicef_results/lessons'
# key='lessons'
# filters=['need to', 'may', 'lesson']
# remove=['recommendations', 'recommend']

sents_num=[]
filtered_num=[]

top_n=30
for fi in files:
  indices_filtered=set()

  df=pd.read_csv(fi).fillna('')
  df.sort_values(by=[key], ascending=[False], inplace=True)#add top n predicted sentences
  indices_predicted=list(df.index.to_list()[:top_n])
  indices_filtered.update(indices_predicted)

  for f in filters:#add top n filtered sentences
    df_f=df[df['comment_text'].str.contains(f)]

    df_f.sort_values(by=[key], ascending=[False], inplace=True)
    indices_predicted=list(df_f.index.to_list()[:top_n])
    indices_filtered.update(indices_predicted)

  print('Obtained {} sentences through prediction and filtering with {} filters out of {} sentences for file {}'.format(len(indices_filtered), len(filters), df.shape[0], fi))
  sents_num.append(df.shape[0])
  filtered_num.append(len(indices_filtered))

  expanded=set()#add following sentence for each of these
  expanded.update(indices_filtered)

  # for i in indices_filtered:
  #   if i+1<df.shape[0]:

  #     expanded.add(i+1)

  df_final=df.loc[list(expanded)].sort_index(axis=0)

  #print(df_final.head())
  print(df_final.shape)

  id=fi.split('/')[-1].replace('.csv', '.txt')
  comments=list(df_final['comment_text'])
  for r in remove:
    comments=[c for c in comments if r not in c]

  print('{} sentences left'.format(len(comments)))
  with open(os.path.join(new_path, id), 'w', encoding='utf-8') as outff:
    outff.writelines(['{}\n'.format(ll[:500]) for ll in comments])

  print(id)

  #print(sents)
import statistics
print(statistics.stdev(sents_num))
print(sum(sents_num)/len(sents_num))

print(statistics.stdev(filtered_num))
print(sum(filtered_num)/len(filtered_num))



In [ ]:
import pandas as pd
from glob import glob
from google.colab import drive
drive.mount('/content/drive')
from glob import glob
import pandas  as pd
import os

files=glob('/content/drive/MyDrive/Colab Notebooks/unicef_results/classified_data/*')

new_path='/content/drive/MyDrive/Colab Notebooks/unicef_results/lesson_recommendation'

key='recommendation'
filters=['should', 'must', 'recommend']#recomemndations
remove=['lesson learned','lesson learnet','Lesson learnet', 'lessons learned', 'Lesson learned', 'Lessons learned']


key2='lessons'
filters2=['need to', 'may', 'lesson']
remove2=['recommendations', 'recommend']

sents_num=[]
filtered_num=[]


#filters=['may', 'need to', 'lesson']#lessons
top_n=30
for fi in files:
  indices_filtered=set()

  df=pd.read_csv(fi).fillna('')
  df.sort_values(by=[key], ascending=[False], inplace=True)#add top n predicted sentences
  indices_predicted=list(df.index.to_list()[:top_n])
  indices_filtered.update(indices_predicted)

  ######################nr 2

  df2=pd.read_csv(fi).fillna('')
  df2.sort_values(by=[key2], ascending=[False], inplace=True)#add top n predicted sentences
  indices_predicted2=list(df2.index.to_list()[:top_n])
  indices_filtered.update(indices_predicted2)

################################################

  for f in filters:#add top n filtered sentences
    df_f=df[df['comment_text'].str.contains(f)]

    df_f.sort_values(by=[key], ascending=[False], inplace=True)
    indices_predicted=list(df_f.index.to_list()[:top_n])
    indices_filtered.update(indices_predicted)

########################################
  for f in filters2:#add top n filtered sentences
    df_f2=df2[df2['comment_text'].str.contains(f)]

    df_f2.sort_values(by=[key2], ascending=[False], inplace=True)
    indices_predicted2=list(df_f2.index.to_list()[:top_n])
    indices_filtered.update(indices_predicted2)


  print('Obtained {} sentences through prediction and filtering with {} filters out of {} sentences for file {}'.format(len(indices_filtered), len(filters), df.shape[0], fi))
  sents_num.append(df.shape[0])
  filtered_num.append(len(indices_filtered))

  expanded=set()#add following sentence for each of these
  expanded.update(indices_filtered)

  # for i in indices_filtered:
  #   if i+1<df.shape[0]:

  #     expanded.add(i+1)

  df_final=df.loc[list(expanded)].sort_index(axis=0)

  #print(df_final.head())
  print(df_final.shape)

  id=fi.split('/')[-1].replace('.csv', '.txt')
  comments=list(df_final['comment_text'])
  for r in remove:
    comments=[c for c in comments if r not in c]

  print('{} sentences left'.format(len(comments)))
  with open(os.path.join(new_path, id), 'w', encoding='utf-8') as outff:
    outff.writelines(['{}\n'.format(ll[:500]) for ll in comments])

  print(id)

  #print(sents)
import statistics
print(statistics.stdev(sents_num))
print(sum(sents_num)/len(sents_num))

print(statistics.stdev(filtered_num))
print(sum(filtered_num)/len(filtered_num))



## Stats

In [ ]:
import statistics
print(statistics.stdev(sents_num))
print(sum(sents_num)/len(sents_num))

print(statistics.stdev(filtered_num))
print(sum(filtered_num)/len(filtered_num))

875.9578920639303
1662.9477303988997
39.0821316069084
143.1292984869326
